In [1]:
import typing
import pandas as pd
from tqdm import tqdm
import config
import src
import requests
import tqdm
import json
import numpy as np
import logging

import cltrier_lib as lib

from sklearn.metrics import cohen_kappa_score, classification_report
import krippendorff
import yaml


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\sstolwi\Git

In [27]:
CFG = config.Config()

In [28]:
CFG.prompt_classify_files.items()

dict_items([('interactivity_acknowledgement', WindowsPath('data/prompts_classify/interactivity_acknowledgement.json')), ('political_ideology', WindowsPath('data/prompts_classify/political_ideology.json')), ('political_ideology_GER', WindowsPath('data/prompts_classify/political_ideology_GER.json')), ('political_ideology_GER2', WindowsPath('data/prompts_classify/political_ideology_GER2.json')), ('political_ideology_GER3', WindowsPath('data/prompts_classify/political_ideology_GER3.json')), ('political_ideology_US', WindowsPath('data/prompts_classify/political_ideology_US.json')), ('political_negativity', WindowsPath('data/prompts_classify/political_negativity.json')), ('rationality_background_info', WindowsPath('data/prompts_classify/rationality_background_info.json')), ('rationality_external_evidence', WindowsPath('data/prompts_classify/rationality_external_evidence.json')), ('rationality_reasoning', WindowsPath('data/prompts_classify/rationality_reasoning.json')), ('rationality_topic_re

In [4]:
pd.set_option('display.max_colwidth', 100) 

In [5]:
MODEL: str = 'llama3.1:70b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'

In [6]:
#load data
MHdata: pd.DataFrame = pd.read_csv('data/MH_BClemm_data/germany_val_all.csv') 
MHdata

,text,pol_label,neg_label,neg3_label,ideo_label,GPT_Pol1,GPT_Pol2,GPT_Neg1,GPT_Neg2,GPT_Neg_3Way1,GPT_Neg_3Way2,GPT_Ideo1,GPT_Ideo2,Pol_Recon,Neg_Recon,Neg3_Recon,Ideo_Recon
0,@thatleechi17 @KlemmChr @AndiScheuer @victorperli Ich würde sagen Sie vergleichen maximal kenntn...,1,1,0,1,1,1,1,1,0,0,1,1,1,1,0,1
1,"Glauben Sie nur nicht, dass Menschen die nicht dem Bundestag angehören, nichts zu sagen hätten. ...",1,0,1,1,1,1,0,0,1,1,1,1,1,0,1,1
2,Das ist ja das Mindeste. \r\n\r\nDass das aber das alles möglich ist trotz der Situation der Uig...,1,1,0,1,1,1,1,1,0,0,0,0,1,1,0,0
3,@Challen81690499 @c_lindner @rbrinkhaus @NowaboFM Na na na... TW,0,0,1,1,1,0,0,0,1,1,1,1,0,0,1,1
4,Ich sage: #DANKE!!!\r\nhttps://t.co/0pVneqXwTC\r\n#afd #btw #btw17 #btw2017,1,0,2,2,1,1,0,0,2,2,2,2,1,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,"@rehbergkk Ihr Denkfehler, in der Tat!",0,1,0,1,0,0,1,1,0,0,1,1,0,1,0,1
696,Die Bundesregierung spricht von „Einwanderung in die Sozialsysteme“ und erteilt gleichzeitig Arb...,1,1,0,0,1,1,0,1,0,0,0,0,1,1,0,0
697,Im #EU Parlament sitzen teilweise echt Verrückte https://t.co/wy2v0pYWLB,1,1,0,1,1,1,1,1,0,0,1,1,1,1,0,1
698,@StBrandner @M_HarderKuehnel Du natürlich! <U+0001F60E><U+0001F600>,0,0,1,1,0,0,0,0,2,2,1,1,0,0,2,1


In [7]:
#change options to low temperature (0,1) and compare result:
options_low = """
seed: 42
temperature: 0.1
"""

options_low = yaml.safe_load(options_low)

In [20]:
#the _label variables are the reconciled and definitive annotation values for the variables
#what does Llama3.1 predict for political orientation?
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (MHdata["text"], MODEL, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label == 'political_ideology'
}

classifying political_ideology: 100%|██████████| 700/700 [49:31<00:00,  4.24s/it]  


In [9]:
predictions #note that these are based on llama3.1:70b-instruct-q6_K

{'political_ideology': 0           neutral
 1           neutral
 2           liberal
 3           neutral
 4      conservative
            ...     
 695         neutral
 696         liberal
 697    conservative
 698         neutral
 699         neutral
 Name: political_ideology, Length: 700, dtype: object}

In [21]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_ideology_low'})
    MHdata = MHdata.join(preds_df)

political_ideology
neutral         474
liberal         145
conservative     81
Name: count, dtype: int64
------------------------------------------


In [11]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_ideology'})
    MHdata = MHdata.join(preds_df)

political_ideology
neutral         454
liberal         159
conservative     87
Name: count, dtype: int64
------------------------------------------


In [23]:
#missing values in low temperature ideology:
MHdata.Llama31_ideology_low.isna().sum()

np.int64(0)

In [52]:
MHdata.Llama31_ideology.isna().sum()

np.int64(0)

In [12]:
#does Llama political ideology differ from manual political ideology ?
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama31_ideology'])

Llama31_ideology,conservative,liberal,neutral
ideo_label,,,
0,4,94,17
1,26,56,421
2,57,9,16


In [24]:
#does Llama political ideology low differ from manual political ideology ?
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama31_ideology_low'])

Llama31_ideology_low,conservative,liberal,neutral
ideo_label,,,
0,3,91,21
1,23,47,433
2,55,7,20


In [16]:
#convert the political ideology labels to numbers:
MHdata['Llama_ideo_score'] = MHdata['Llama31_ideology'].map({'liberal': 0, 'neutral': 1, 'conservative': 2})

In [25]:
MHdata['Llama_ideo_score_low'] = MHdata['Llama31_ideology_low'].map({'liberal': 0, 'neutral': 1, 'conservative': 2})

In [17]:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama_ideo_score'])

Llama_ideo_score,0,1,2
ideo_label,,,
0,94,17,4
1,56,421,26
2,9,16,57


In [18]:
#and how does Llama3.1 perform compared to GPT
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'GPT_Ideo1'], MHdata.loc[:, 'Llama_ideo_score'])

Llama_ideo_score,0,1,2
GPT_Ideo1,,,
0,115,49,11
1,29,381,13
2,15,24,63


In [19]:
llm_human_column_pairs = [
    ("ideo_label", "Llama_ideo_score"),
    ("ideo_label", "GPT_Ideo1")
# ("TopicRelevance", "rationality_topic_relevance"),
]

In [20]:
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdata[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.59      0.82      0.69       115
           1       0.93      0.84      0.88       503
           2       0.66      0.70      0.67        82

    accuracy                           0.82       700
   macro avg       0.72      0.78      0.75       700
weighted avg       0.84      0.82      0.82       700

cohen_kappa_score(ideo_label): 0.6206909689737066
krippendorf(ideo_label): 0.6192039671924076
---
              precision    recall  f1-score   support

           0       0.61      0.92      0.73       115
           1       0.96      0.81      0.88       503
           2       0.69      0.85      0.76        82

    accuracy                           0.83       700
   macro avg       0.75      0.86      0.79       700
weighted avg       0.87      0.83      0.84       700

cohen_kappa_score(ideo_label): 

In [ ]:
#Llama did ok, but GPT4 did better.

In [26]:
llm_human_column_pairs = [
    ("ideo_label", "Llama_ideo_score_low"),
#   ("ideo_label", "GPT_Ideo1")
# ("TopicRelevance", "rationality_topic_relevance"),
]
print("low temperature Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdata[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

low temperature Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.63      0.79      0.70       115
           1       0.91      0.86      0.89       503
           2       0.68      0.67      0.67        82

    accuracy                           0.83       700
   macro avg       0.74      0.77      0.75       700
weighted avg       0.84      0.83      0.83       700

cohen_kappa_score(ideo_label): 0.6289335453713074
krippendorf(ideo_label): 0.6284897871397188


#very similar performance for low temperature english prompt of Llama3.1 compared to default temperature

In [8]:
#what if we translate the prompt first 
#what does Llama3.1 predict for political orientation?
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (MHdata["text"], model=MODEL, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label == 'political_ideology_GER'
}

classifying political_ideology_GER: 100%|██████████| 700/700 [06:46<00:00,  1.72it/s]


In [9]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_ideo_GER_low'})
    MHdata = MHdata.join(preds_df)

political_ideology_GER
Mitte     564
Links      83
Rechts     38
Name: count, dtype: int64
------------------------------------------


In [58]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_ideo_GER'})
    MHdata = MHdata.join(preds_df)

political_ideology_GER
Mitte     505
Links      97
Rechts     48
Name: count, dtype: int64
------------------------------------------


In [ ]:
MHdata.Llama31_ideo_GER.isna().sum()    

In [11]:
MHdata.Llama31_ideo_GER_low.isna().sum()    

np.int64(15)

In [61]:
#convert the political ideology labels to numbers:
MHdata['Llama_ideoGER_score'] = MHdata['Llama31_ideo_GER'].map({'Links': 0, 'Mitte': 1, 'Rechts': 2})

In [12]:
#convert the political ideology labels to numbers:
MHdata['Llama_ideoGER_score_low'] = MHdata['Llama31_ideo_GER_low'].map({'Links': 0, 'Mitte': 1, 'Rechts': 2})

In [62]:
#does Llama political ideology differ from manual political ideology ?
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama_ideoGER_score'])

Llama_ideoGER_score,0.0,1.0,2.0
ideo_label,,,
0,59,51,0
1,32,421,11
2,6,33,37


In [13]:
#does Llama political ideology differ from manual political ideology ?
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama_ideoGER_score_low'])

Llama_ideoGER_score_low,0.0,1.0,2.0
ideo_label,,,
0,62,51,0
1,20,474,3
2,1,39,35


In [16]:
62+474+35

571

In [17]:
564+83+38


685

In [19]:
#percent agreement ideoGER_low:
100*571/685

83.35766423357664

In [63]:
#and compared to the english prompt:
pd.crosstab(MHdata.loc[:, 'Llama31_ideology'], MHdata.loc[:, 'Llama31_ideo_GER'])

Llama31_ideo_GER,Links,Mitte,Rechts
Llama31_ideology,,,
conservative,8,26,43
liberal,70,80,0
neutral,19,399,5


In [64]:
llm_human_column_pairs = [
    ("ideo_label", "Llama_ideoGER_score"),
    ("Llama_ideo_score", "Llama_ideoGER_score")
# ("TopicRelevance", "rationality_topic_relevance"),
]

In [14]:
llm_human_column_pairs = [
    ("ideo_label", "Llama_ideoGER_score_low"),
# ("TopicRelevance", "rationality_topic_relevance"),
]

In [15]:
print("Options low: Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdata[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Options low: Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.75      0.55      0.63       113
           1       0.84      0.95      0.89       497
           2       0.92      0.47      0.62        75

    accuracy                           0.83       685
   macro avg       0.84      0.66      0.72       685
weighted avg       0.83      0.83      0.82       685

cohen_kappa_score(ideo_label): 0.5580345014941591
krippendorf(ideo_label): 0.5540933207998926


considerably better reliability and less missing values than without low temperature setting, but still not as good as the english prompt

In [65]:
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdata[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.61      0.54      0.57       110
           1       0.83      0.91      0.87       464
           2       0.77      0.49      0.60        76

    accuracy                           0.80       650
   macro avg       0.74      0.64      0.68       650
weighted avg       0.79      0.80      0.79       650

cohen_kappa_score(ideo_label): 0.5027665619859429
krippendorf(ideo_label): 0.5012600784626177
---
              precision    recall  f1-score   support

           0       0.72      0.47      0.57       150
           1       0.79      0.94      0.86       423
           2       0.90      0.56      0.69        77

    accuracy                           0.79       650
   macro avg       0.80      0.66      0.70       650
weighted avg       0.79      0.79      0.77       650

cohen_kappa_score(Llama_ideo_sc

In [ ]:
#unexpectedly, the german prompt did much worse than the english prompt.

In [66]:
#save data to parquet
MHdata.to_parquet('data/MH_BClemm_data/germany_val_all_llama.parquet')

In [41]:
#what if we adapt ideology labels further:
#what does Llama3.1 predict for political orientation?
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (MHdata["text"], MODEL)
    )
    for label, path in CFG.prompt_classify_files.items() if label == 'political_ideology_GER2'
}

classifying political_ideology_GER: 100%|██████████| 700/700 [12:55<00:00,  1.11s/it]


In [42]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_ideo_GER2'})
    MHdata = MHdata.join(preds_df)

political_ideology_GER
Mittlere Mitte    420
Linksliberal      148
Konservativ        46
Name: count, dtype: int64
------------------------------------------


In [45]:
#convert the political ideology labels to numbers:
MHdata['Llama_ideoGER_score2'] = MHdata['Llama31_ideo_GER2'].map({'Linksliberal': 0, 'Mittlere Mitte': 1, 'Konservativ': 2})

In [46]:
MHdata['Llama_ideoGER_score2'].isna().sum()

np.int64(86)

In [47]:
#does Llama political ideology differ from manual political ideology ?
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama_ideoGER_score2'])

Llama_ideoGER_score2,0.0,1.0,2.0
ideo_label,,,
0,75,29,4
1,60,370,13
2,13,21,29


In [67]:
#and compared to the GER1 prompt:
pd.crosstab(MHdata.loc[:, 'Llama31_ideo_GER'], MHdata.loc[:, 'Llama31_ideo_GER2'])

Llama31_ideo_GER2,Konservativ,Linksliberal,Mittlere Mitte
Llama31_ideo_GER,,,
Links,7,63,20
Mitte,11,72,375
Rechts,27,3,5


In [68]:
llm_human_column_pairs = [
    ("ideo_label", "Llama_ideoGER_score2"),
    ("Llama_ideoGER_score", "Llama_ideoGER_score2")
# ("TopicRelevance", "rationality_topic_relevance"),
]

In [69]:
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdata[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.51      0.69      0.59       108
           1       0.88      0.84      0.86       443
           2       0.63      0.46      0.53        63

    accuracy                           0.77       614
   macro avg       0.67      0.66      0.66       614
weighted avg       0.79      0.77      0.78       614

cohen_kappa_score(ideo_label): 0.5003894126262686
krippendorf(ideo_label): 0.4990361593355478
---
              precision    recall  f1-score   support

         0.0       0.46      0.70      0.55        90
         1.0       0.94      0.82      0.87       458
         2.0       0.60      0.77      0.68        35

    accuracy                           0.80       583
   macro avg       0.66      0.76      0.70       583
weighted avg       0.84      0.80      0.81       583

cohen_kappa_score(Llama_ideoGER

In [ ]:
#No improvement of performance.

In [51]:
#save data to parquet
MHdata.to_parquet('data/MH_BClemm_data/germany_val_all_llama.parquet')

In [29]:
#final try to improve German language prompt:
#what does Llama3.1 predict for political orientation?
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (MHdata["text"], MODEL, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label == 'political_ideology_GER3'
}

classifying political_ideology_GER3: 100%|██████████| 700/700 [42:05<00:00,  3.61s/it]  


In [30]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to 'Llama31_ideology'
    preds_df = preds_df.rename(columns={preds_df.columns[0]: 'Llama31_ideo_GER3'})
    MHdata = MHdata.join(preds_df)

political_ideology_GER3
Moderate       513
Liberal         87
Konservativ     45
Name: count, dtype: int64
------------------------------------------


In [32]:
#convert the political ideology labels to numbers:
MHdata['Llama_ideoGER_score3'] = MHdata['Llama31_ideo_GER3'].map({'Liberal': 0, 'Moderate': 1, 'Konservativ': 2})

In [33]:
MHdata['Llama_ideoGER_score3'].isna().sum()

np.int64(55)

In [34]:
#does Llama political ideology differ from manual political ideology ?
#make a crosstab to see if there are differences:
pd.crosstab(MHdata.loc[:, 'ideo_label'], MHdata.loc[:, 'Llama_ideoGER_score3'])

Llama_ideoGER_score3,0.0,1.0,2.0
ideo_label,,,
0,59,46,0
1,23,441,9
2,5,26,36


In [35]:
llm_human_column_pairs = [
    ("ideo_label", "Llama_ideoGER_score3"),
#   ("Llama_ideoGER_score", "Llama_ideoGER_score2")
# ("TopicRelevance", "rationality_topic_relevance"),
]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = MHdata[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.68      0.56      0.61       105
           1       0.86      0.93      0.89       473
           2       0.80      0.54      0.64        67

    accuracy                           0.83       645
   macro avg       0.78      0.68      0.72       645
weighted avg       0.82      0.83      0.82       645

cohen_kappa_score(ideo_label): 0.5639350973168099
krippendorf(ideo_label): 0.562640078693097


Best performing German Llama3.1 model so far, but did not reduce (but increased) number of missing classifications, and still performance is inferior to english language prompt

In [ ]:
#join to parquet data:
#save data to parquet
MHdatapar= pd.read_parquet('data/MH_BClemm_data/germany_val_all_llama.parquet')

In [42]:
MHdatapar.head()

,text,pol_label,neg_label,neg3_label,ideo_label,GPT_Pol1,GPT_Pol2,GPT_Neg1,GPT_Neg2,GPT_Neg_3Way1,...,Llama31_ideo_GER,Llama_ideoGER_score,Llama31_ideo_GER2,Llama_ideoGER_score2,Llama31_ideo_GER3,Llama_ideoGER_score3,Llama31_ideology_low,Llama_ideo_score_low,Llama31_ideo_GER_low,Llama_ideoGER_score_low
0,@thatleechi17 @KlemmChr @AndiScheuer @victorperli Ich würde sagen Sie vergleichen maximal kenntn...,1,1,0,1,1,1,1,1,0,...,Mitte,1.0,Mittlere Mitte,1.0,Moderate,1.0,neutral,1,Mitte,1.0
1,"Glauben Sie nur nicht, dass Menschen die nicht dem Bundestag angehören, nichts zu sagen hätten. ...",1,0,1,1,1,1,0,0,1,...,Mitte,1.0,Mittlere Mitte,1.0,Moderate,1.0,liberal,0,Mitte,1.0
2,Das ist ja das Mindeste. \r\n\r\nDass das aber das alles möglich ist trotz der Situation der Uig...,1,1,0,1,1,1,1,1,0,...,Mitte,1.0,Linksliberal,0.0,Moderate,1.0,liberal,0,Links,0.0
3,@Challen81690499 @c_lindner @rbrinkhaus @NowaboFM Na na na... TW,0,0,1,1,1,0,0,0,1,...,Mitte,1.0,Mittlere Mitte,1.0,Moderate,1.0,neutral,1,Mitte,1.0
4,Ich sage: #DANKE!!!\r\nhttps://t.co/0pVneqXwTC\r\n#afd #btw #btw17 #btw2017,1,0,2,2,1,1,0,0,2,...,Rechts,2.0,None,NaN,Konservativ,2.0,conservative,2,Rechts,2.0


In [41]:
MHdatapar = MHdatapar.join(MHdata.loc[:, ['Llama31_ideo_GER3', 'Llama_ideoGER_score3', 'Llama31_ideology_low', 'Llama_ideo_score_low', 'Llama31_ideo_GER_low' ,'Llama_ideoGER_score_low']])   


In [43]:
MHdatapar.to_parquet('data/MH_BClemm_data/germany_val_all_llama.parquet')